In [1]:
import pandas as pd
from utils import RBKPreprocessor, RBKRegressor, local_metric, write_predictions, get_mean_predictions
from sklearn.model_selection import train_test_split, KFold
from catboost import CatBoostRegressor, Pool



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
RANDOM_STATE = 44

In [3]:
train_df = pd.read_csv("data/train_dataset_train.csv", index_col=0)
test_df = pd.read_csv("data/test_dataset_test.csv", index_col=0)

train_df["is_train"] = 1
test_df["is_train"] = 0

df = pd.concat([train_df, test_df])

## Натренируем препроцессор, который подготовим нам данные для обучения модели

In [4]:
%%time
preprocessor = RBKPreprocessor()
preprocessor.fit(df)


Wall time: 4.07 s


RBKPreprocessor()

In [ ]:
## Используем свой класс Регрессор

Который просто скрывает логику тренировки отдельных CatBoostRegressor для каждой переменной
Попробуем натренировать на 4 фолдах модели, а потом усреднить их предсказания

In [5]:
splitter = KFold(n_splits=4, random_state=RANDOM_STATE, shuffle=True)

In [6]:
my_models = []
for train_index, test_index in splitter.split(train_df):
    X_train = preprocessor.transform(train_df.iloc[train_index])
    y_train = train_df.iloc[train_index][["views", "depth", "full_reads_percent"]]

    X_test = preprocessor.transform(train_df.iloc[test_index])
    y_test = train_df.iloc[test_index][["views", "depth", "full_reads_percent"]]

    model = RBKRegressor(random_seed=RANDOM_STATE, n_iterations=1000)
    model.fit(X_train, y_train, X_test, y_test)
    
    my_models.append(model)


C:\Soft\Anaconda\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Soft\Anaconda\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


0:	learn: 0.0277375	test: 0.0100285	best: 0.0100285 (0)	total: 325ms	remaining: 5m 24s
1:	learn: 0.0523555	test: 0.0199962	best: 0.0199962 (1)	total: 512ms	remaining: 4m 15s
2:	learn: 0.0872904	test: 0.0295954	best: 0.0295954 (2)	total: 646ms	remaining: 3m 34s
3:	learn: 0.1107828	test: 0.0386803	best: 0.0386803 (3)	total: 805ms	remaining: 3m 20s
4:	learn: 0.1431485	test: 0.0481891	best: 0.0481891 (4)	total: 968ms	remaining: 3m 12s
5:	learn: 0.1653301	test: 0.0657559	best: 0.0657559 (5)	total: 1.17s	remaining: 3m 13s
6:	learn: 0.1928192	test: 0.0737108	best: 0.0737108 (6)	total: 1.34s	remaining: 3m 10s
7:	learn: 0.2112380	test: 0.0811761	best: 0.0811761 (7)	total: 1.5s	remaining: 3m 6s
8:	learn: 0.2287464	test: 0.0882696	best: 0.0882696 (8)	total: 1.7s	remaining: 3m 7s
9:	learn: 0.2842683	test: 0.1216308	best: 0.1216308 (9)	total: 1.85s	remaining: 3m 3s
10:	learn: 0.2998987	test: 0.1280886	best: 0.1280886 (10)	total: 1.98s	remaining: 2m 57s
11:	learn: 0.3448923	test: 0.1596414	best: 0.1

94:	learn: 0.8519936	test: 0.4817438	best: 0.4817438 (94)	total: 15s	remaining: 2m 22s
95:	learn: 0.8532140	test: 0.4818696	best: 0.4818696 (95)	total: 15.2s	remaining: 2m 22s
96:	learn: 0.8558771	test: 0.4864543	best: 0.4864543 (96)	total: 15.4s	remaining: 2m 23s
97:	learn: 0.8565683	test: 0.4864118	best: 0.4864543 (96)	total: 15.5s	remaining: 2m 23s
98:	learn: 0.8573792	test: 0.4863424	best: 0.4864543 (96)	total: 15.7s	remaining: 2m 23s
99:	learn: 0.8591699	test: 0.4869641	best: 0.4869641 (99)	total: 15.9s	remaining: 2m 23s
100:	learn: 0.8598132	test: 0.4869920	best: 0.4869920 (100)	total: 16.1s	remaining: 2m 22s
101:	learn: 0.8604458	test: 0.4871063	best: 0.4871063 (101)	total: 16.2s	remaining: 2m 22s
102:	learn: 0.8618598	test: 0.4872639	best: 0.4872639 (102)	total: 16.4s	remaining: 2m 22s
103:	learn: 0.8624442	test: 0.4871976	best: 0.4872639 (102)	total: 16.5s	remaining: 2m 22s
104:	learn: 0.8630336	test: 0.4871105	best: 0.4872639 (102)	total: 16.8s	remaining: 2m 22s
105:	learn: 0

185:	learn: 0.9101396	test: 0.5010597	best: 0.5017735 (181)	total: 37.8s	remaining: 2m 45s
186:	learn: 0.9105275	test: 0.5010333	best: 0.5017735 (181)	total: 38.1s	remaining: 2m 45s
187:	learn: 0.9107137	test: 0.5012681	best: 0.5017735 (181)	total: 38.3s	remaining: 2m 45s
188:	learn: 0.9109105	test: 0.5012905	best: 0.5017735 (181)	total: 38.6s	remaining: 2m 45s
189:	learn: 0.9121185	test: 0.5021931	best: 0.5021931 (189)	total: 38.9s	remaining: 2m 45s
190:	learn: 0.9123076	test: 0.5021983	best: 0.5021983 (190)	total: 39.2s	remaining: 2m 45s
191:	learn: 0.9124965	test: 0.5021947	best: 0.5021983 (190)	total: 39.4s	remaining: 2m 45s
192:	learn: 0.9137849	test: 0.5025490	best: 0.5025490 (192)	total: 39.7s	remaining: 2m 46s
193:	learn: 0.9140531	test: 0.5025375	best: 0.5025490 (192)	total: 40s	remaining: 2m 46s
194:	learn: 0.9142232	test: 0.5025580	best: 0.5025580 (194)	total: 40.3s	remaining: 2m 46s
195:	learn: 0.9144013	test: 0.5025200	best: 0.5025580 (194)	total: 40.5s	remaining: 2m 46s
1

276:	learn: 0.9326454	test: 0.5072020	best: 0.5072131 (275)	total: 1m 4s	remaining: 2m 48s
277:	learn: 0.9327488	test: 0.5071561	best: 0.5072131 (275)	total: 1m 4s	remaining: 2m 48s
278:	learn: 0.9329043	test: 0.5071553	best: 0.5072131 (275)	total: 1m 5s	remaining: 2m 48s
279:	learn: 0.9332377	test: 0.5066828	best: 0.5072131 (275)	total: 1m 5s	remaining: 2m 47s
280:	learn: 0.9333397	test: 0.5066808	best: 0.5072131 (275)	total: 1m 5s	remaining: 2m 47s
281:	learn: 0.9334436	test: 0.5066807	best: 0.5072131 (275)	total: 1m 5s	remaining: 2m 47s
282:	learn: 0.9338141	test: 0.5068379	best: 0.5072131 (275)	total: 1m 6s	remaining: 2m 47s
283:	learn: 0.9339174	test: 0.5068083	best: 0.5072131 (275)	total: 1m 6s	remaining: 2m 47s
284:	learn: 0.9340177	test: 0.5067997	best: 0.5072131 (275)	total: 1m 6s	remaining: 2m 47s
285:	learn: 0.9341191	test: 0.5068064	best: 0.5072131 (275)	total: 1m 6s	remaining: 2m 47s
286:	learn: 0.9345453	test: 0.5069647	best: 0.5072131 (275)	total: 1m 7s	remaining: 2m 47s

...

520:	learn: 0.9540383	test: 0.7534200	best: 0.7536031 (516)	total: 7m 56s	remaining: 7m 18s
521:	learn: 0.9540919	test: 0.7533954	best: 0.7536031 (516)	total: 7m 57s	remaining: 7m 17s
522:	learn: 0.9541387	test: 0.7533824	best: 0.7536031 (516)	total: 7m 58s	remaining: 7m 16s
523:	learn: 0.9542267	test: 0.7534313	best: 0.7536031 (516)	total: 7m 59s	remaining: 7m 15s
524:	learn: 0.9543020	test: 0.7535357	best: 0.7536031 (516)	total: 8m	remaining: 7m 14s
525:	learn: 0.9543590	test: 0.7535827	best: 0.7536031 (516)	total: 8m 1s	remaining: 7m 13s
526:	learn: 0.9545202	test: 0.7535328	best: 0.7536031 (516)	total: 8m 1s	remaining: 7m 12s
527:	learn: 0.9546593	test: 0.7535165	best: 0.7536031 (516)	total: 8m 2s	remaining: 7m 11s
528:	learn: 0.9547126	test: 0.7536211	best: 0.7536211 (528)	total: 8m 3s	remaining: 7m 10s
529:	learn: 0.9547480	test: 0.7536203	best: 0.7536211 (528)	total: 8m 4s	remaining: 7m 9s
530:	learn: 0.9547960	test: 0.7536208	best: 0.7536211 (528)	total: 8m 5s	remaining: 7m 8s


610:	learn: 0.9603378	test: 0.7546545	best: 0.7546799 (607)	total: 9m 18s	remaining: 5m 55s
611:	learn: 0.9603748	test: 0.7546774	best: 0.7546799 (607)	total: 9m 19s	remaining: 5m 54s
612:	learn: 0.9604622	test: 0.7547160	best: 0.7547160 (612)	total: 9m 20s	remaining: 5m 53s
613:	learn: 0.9605062	test: 0.7547282	best: 0.7547282 (613)	total: 9m 21s	remaining: 5m 53s
614:	learn: 0.9606317	test: 0.7547348	best: 0.7547348 (614)	total: 9m 22s	remaining: 5m 52s
615:	learn: 0.9606780	test: 0.7547371	best: 0.7547371 (615)	total: 9m 23s	remaining: 5m 51s
616:	learn: 0.9607122	test: 0.7547296	best: 0.7547371 (615)	total: 9m 24s	remaining: 5m 50s
617:	learn: 0.9607577	test: 0.7547152	best: 0.7547371 (615)	total: 9m 25s	remaining: 5m 49s
618:	learn: 0.9607866	test: 0.7547266	best: 0.7547371 (615)	total: 9m 26s	remaining: 5m 48s
619:	learn: 0.9608160	test: 0.7547127	best: 0.7547371 (615)	total: 9m 27s	remaining: 5m 47s
620:	learn: 0.9608646	test: 0.7547764	best: 0.7547764 (620)	total: 9m 28s	remain

7:	learn: 0.2337096	test: 0.1793028	best: 0.1800250 (6)	total: 9.65s	remaining: 19m 57s
8:	learn: 0.2374963	test: 0.1792788	best: 0.1800250 (6)	total: 10.9s	remaining: 19m 58s
9:	learn: 0.2410892	test: 0.1799524	best: 0.1800250 (6)	total: 12.1s	remaining: 20m 2s
10:	learn: 0.2444904	test: 0.1791023	best: 0.1800250 (6)	total: 13.3s	remaining: 19m 59s
11:	learn: 0.2574497	test: 0.1895157	best: 0.1895157 (11)	total: 14.5s	remaining: 19m 55s
12:	learn: 0.2603543	test: 0.1896118	best: 0.1896118 (12)	total: 15.8s	remaining: 20m 1s
13:	learn: 0.2688782	test: 0.1950926	best: 0.1950926 (13)	total: 17s	remaining: 19m 58s
14:	learn: 0.2715536	test: 0.1952092	best: 0.1952092 (14)	total: 18.1s	remaining: 19m 50s
15:	learn: 0.2738771	test: 0.1952283	best: 0.1952283 (15)	total: 19.4s	remaining: 19m 55s
16:	learn: 0.2826026	test: 0.2019326	best: 0.2019326 (16)	total: 20.6s	remaining: 19m 51s
17:	learn: 0.2846048	test: 0.2018085	best: 0.2019326 (16)	total: 21.9s	remaining: 19m 53s
18:	learn: 0.2933168	

98:	learn: 0.5597989	test: 0.3619406	best: 0.3619406 (98)	total: 1m 49s	remaining: 16m 37s
99:	learn: 0.5614987	test: 0.3619182	best: 0.3619406 (98)	total: 1m 50s	remaining: 16m 35s
100:	learn: 0.5634207	test: 0.3623274	best: 0.3623274 (100)	total: 1m 51s	remaining: 16m 34s
101:	learn: 0.5646982	test: 0.3625022	best: 0.3625022 (101)	total: 1m 52s	remaining: 16m 33s
102:	learn: 0.5667666	test: 0.3629101	best: 0.3629101 (102)	total: 1m 53s	remaining: 16m 31s
103:	learn: 0.5685145	test: 0.3625308	best: 0.3629101 (102)	total: 1m 54s	remaining: 16m 29s
104:	learn: 0.5690797	test: 0.3621005	best: 0.3629101 (102)	total: 1m 55s	remaining: 16m 27s
105:	learn: 0.5709807	test: 0.3623609	best: 0.3629101 (102)	total: 1m 57s	remaining: 16m 27s
106:	learn: 0.5725137	test: 0.3624262	best: 0.3629101 (102)	total: 1m 58s	remaining: 16m 25s
107:	learn: 0.5736795	test: 0.3628259	best: 0.3629101 (102)	total: 1m 59s	remaining: 16m 23s
108:	learn: 0.5751763	test: 0.3632370	best: 0.3632370 (108)	total: 1m 59s	

187:	learn: 0.6654480	test: 0.3760203	best: 0.3773869 (173)	total: 3m 16s	remaining: 14m 8s
188:	learn: 0.6659711	test: 0.3760276	best: 0.3773869 (173)	total: 3m 17s	remaining: 14m 6s
189:	learn: 0.6665936	test: 0.3762261	best: 0.3773869 (173)	total: 3m 18s	remaining: 14m 5s
190:	learn: 0.6673316	test: 0.3763063	best: 0.3773869 (173)	total: 3m 18s	remaining: 14m 2s
191:	learn: 0.6680167	test: 0.3763366	best: 0.3773869 (173)	total: 3m 19s	remaining: 14m
192:	learn: 0.6690136	test: 0.3766345	best: 0.3773869 (173)	total: 3m 20s	remaining: 13m 58s
193:	learn: 0.6695378	test: 0.3763799	best: 0.3773869 (173)	total: 3m 21s	remaining: 13m 56s
194:	learn: 0.6704902	test: 0.3762141	best: 0.3773869 (173)	total: 3m 22s	remaining: 13m 54s
195:	learn: 0.6709591	test: 0.3761763	best: 0.3773869 (173)	total: 3m 22s	remaining: 13m 52s
196:	learn: 0.6712931	test: 0.3760443	best: 0.3773869 (173)	total: 3m 23s	remaining: 13m 50s
197:	learn: 0.6719421	test: 0.3762229	best: 0.3773869 (173)	total: 3m 24s	rema

## Оценка на обучении. По моделям и средняя

In [7]:
X_train = preprocessor.transform(train_df)
y_train = train_df[["views", "depth", "full_reads_percent"]]


C:\Soft\Anaconda\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
for model in my_models:
    print(local_metric(y_train, model.predict(X_train)))

(0.7882091764147028, 0.2983433326546729, 0.26849983486078255, 0.22136600889924732)
(0.8231959125907111, 0.34750425197975565, 0.2692170182329642, 0.20647464237799132)
(0.8190823253651975, 0.3577023734484537, 0.2695018482044352, 0.19187810371230876)
(0.7544934100771286, 0.31070438763547625, 0.26974827082935554, 0.17404075161229682)


In [9]:
print(local_metric(y_train, get_mean_predictions(my_models, X_train)))

(0.847544675567384, 0.36092379726769536, 0.2746464513672954, 0.21197442693239318)


## Сделаем предсказания на тестовых данных и запишем предикт

In [10]:
X_test = preprocessor.transform(test_df)

C:\Soft\Anaconda\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
preds = get_mean_predictions(my_models, X_test)

In [12]:
write_predictions(preds)

## Результат на лидерборде 0.685924